fatality rate?  total infections? total deaths?


model: fatalities?  (estimate from curves.. when do get peak, how much time has elapsed?)
infection rate?

data:
+ https://github.com/govex/COVID-19
+ https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
+ https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv (used by Johns Hopkins; county-level)


In [2]:
import pandas as pd
from datetime import date

In [3]:
data = pd.read_csv("data/combined_data.csv")

In [4]:
data.head()

,Unnamed: 0,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,ten plus cases
0,0,2.56,580,6009,10431,4473,11938,5507,53742,9.9,...,0,0,0,0,0,0,0.0,0,0,3/28/20
1,1,3.15,7797,3901,11372,4707,3752,4899,52644,15.6,...,0,0,0,0,1,1,1.0,1,1,4/12/20
2,2,3.31,30494,19232,37518,20736,12420,17109,190707,10.5,...,2,2,3,3,3,3,3.0,3,3,3/21/20
3,3,2.37,596,7285,9199,5133,1562,4717,44505,5.1,...,1,1,1,1,1,1,1.0,1,1,3/28/20
4,4,2.59,743,6543,17734,7999,3077,8953,69649,9.5,...,3,3,3,3,3,3,3.0,3,3,4/3/20
